## Apagando dados nulos

https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.DataFrame.dropna.html

Nesta seção, vamos aprender como remover dados nulos de um DataFrame utilizando o PySpark. Dados nulos podem impactar negativamente a análise e a qualidade dos resultados, por isso é importante saber como identificá-los e eliminá-los de forma eficiente. Utilizaremos o método `dropna` para realizar essa tarefa.

`DataFrame.dropna(how='any', thresh=None, subset=None)`

**dropna()**: Se usar assim sem critérios ele apaga todas a linhas se em alguma delas tiver dados nulos. Obs: tenho 40 colunas e em qualquer 1 das colunas eu tiver dados nulos ele vai apagar toda a linha mesmo que as outras colunas nao tenha dado nulos

* **how='any'**: Significa que uma linha será removida se ela contiver pelo menos um valor nulo em qualquer uma de suas colunas.

* **how='all'**: Significa que uma linha será removida somente se todos os seus valores forem nulos.

* **thresh**: A linha será mantida se ela tiver pelo menos x valores não nulos. Por exemplo, se thresh=2, a linha mantida se ela tiver pelo menos 2 valores não nulos. Se tiver menos de 2 valores não nulos, a linha será removida.

* **subset**: Se especificado, o método **dropna()** remove as linhas que contêm valores nulos apenas nas colunas listadas em subset. Por exemplo, se `subset=['coluna1', 'coluna2']`, uma linha será removida se tiver valores nulos apenas em 'coluna1' e 'coluna2', enquanto outros valores nulos em diferentes colunas não afetarão a remoção.

In [0]:
df = spark.read.json("/Volumes/workspace/default/tutorial/anac/V_OCORRENCIA_AMPLA.json")
display(df)

In [0]:
# Salvando em um novo df sem valores nulos
sem_nulo = df.dropna()
display(sem_nulo)

### Usando parametros

In [0]:
# how='all' significa que uma linha será removida  se tiver nulos em todas as linhas 
# how='any' significa que uma linha será removida se ela contiver pelo menos um valor nulo em qualquer uma de suas colunas.
# obs 'any' é a mesma coisa que df.dropna()
display(df.dropna(how='all'))

In [0]:
# thresh: A linha será mantida se ela tiver pelo menos 30 valores não nulos. Se tiver menos de 30 valores não nulos, a linha será removida.
display(df.dropna(thresh=30))

In [0]:
# subset: Por exemplo, se subset=['coluna1', 'coluna2'], se tiver dados nulos nessas colunas de referencia a linha será apagada se nessa coluna nao tiver nulo por mais que as outras tenham os dados nao serao removidos
# Exemplo vou realizar analise de Aerodromo_de_Origem o que for nulo pra min nao importa entao posso passar fogo nos que estao nulos por que vao avacalhar minha minha base

display(df.dropna(subset='Aerodromo_de_Origem'))

In [0]:
# 2° exemplo vou realizar analise de Aerodromo_de_Origem e Danos_a_Aeronave se no tem registro em qualquer um desses 2 o restante dos dados pra min nao importa bora passar fogo tambem 
# Obs como sao dados de 2 colunas ou mais tem que passar em uma lista entre []
display(df.dropna(subset=['Aerodromo_de_Origem', 'Danos_a_Aeronave']))

In [0]:
print(df.columns)

In [0]:
# Exemplo 3 passando fogo em nulos de varias colunas, vamos fazer chique com lista de colunas em variaveis 

colunas_ref=['Regiao','Aerodromo_de_Origem','Municipio','Numero_da_Ocorrencia','Danos_a_Aeronave']
display(df.dropna(subset=colunas_ref))

In [0]:
# Salvando Dados em um novo df

df_tratado= df.dropna(subset=['Aerodromo_de_Origem','Danos_a_Aeronave'])
display(df_tratado)

### Filtrando dados nulos

In [0]:
# Filtrando dados nulos 
display(df.where(df.Aerodromo_de_Destino.isNull()))

In [0]:
# Filtrando dados nao nulos 
display(df.where(df.Aerodromo_de_Destino.isNotNull()))

In [0]:
# Salvando em um novo df
destino_nao_nulo = df.where(df.Aerodromo_de_Destino.isNotNull()) 
display(destino_nao_nulo)

In [0]:
# Contando nulo em coluna 
display(df.where(df.Aerodromo_de_Destino.isNull()).count())

In [0]:
print(df.columns)

In [0]:
coluna = 'Operacao'
display(df.where(df[coluna].isNull()).count())

In [0]:
lista_contagem_nulos = [(coluna, df.where(df[coluna].isNull()).count()) for coluna in df.columns]
display(lista_contagem_nulos)


### Substituindo dados nulos

* **dropna** = apagar nulos 
* **fillna** = Substituir nulo 

In [0]:
coluna = 'Aerodromo_de_Origem'
display(df.where(df[coluna].isNull()).count())

In [0]:
# Substituindo dados nulos de uma coluna especifica (se nao passar a coluna no subset ele faz para todas as colunas )
display(df.fillna('Sem Origem', subset=['Aerodromo_de_Origem']))

In [0]:
# Sem subset = substitui tudos nulos sem criterio 
display(df.fillna('N/A'))

In [0]:
df.printSchema()

In [0]:
# Substituindo dados nulos de uma coluna especifica (atenção com formato de dados)
teste= df.fillna("0", subset=['Lesoes_Desconhecidas_Passageiros'])
display(teste)

In [0]:
# Substitui por 0 em string primeiro e depois converter a coluna pra int (depois vamos fazer tudo junto)
teste = teste.withColumn('Lesoes_Desconhecidas_Passageiros', teste.Lesoes_Desconhecidas_Passageiros.cast("int"))
display(teste)

In [0]:
teste.printSchema()

In [0]:
# Fazendo Conversoes usando withColumn com fillna (fazer o mesmo com as outras colunas de lesoes)
novo = df \
    .withColumn(
        "Lesoes_Desconhecidas_Terceiros",
        df.Lesoes_Desconhecidas_Terceiros.cast("int")
    ) \
    .fillna(0, subset=["Lesoes_Desconhecidas_Terceiros"])

display(novo)

### Tratando nulos SQL

In [0]:
df.createOrReplaceTempView("anac")

In [0]:
%sql
select * from anac

In [0]:
%sql
-- Comando is null e is not null em filtro 
select * from anac
where Aerodromo_de_Origem is null

In [0]:
%sql
-- ver descições da tabela 
DESCRIBE anac

In [0]:
%sql
-- exibir nome das colunas  
SHOW COLUMNS FROM anac


In [0]:
%sql 
-- tratando nulos com método coalesce e ifnull 
select 
    coalesce(Aerodromo_de_Destino, "Sem Registro") as Aerodromo_de_Destino,
    ifnull(Aerodromo_de_Origem, "Sem Origem") as Aerodromo_de_Origem,
    ifnull(CLS, "Sem Registro") as CLSTeste,
    Categoria_da_Aeronave,
    Danos_a_Aeronave,
    Data_da_Ocorrencia,
    UF
from anac

In [0]:
# Salvando consulta tratada em um Df novo
novodf= spark.sql("""
select 
    coalesce(Aerodromo_de_Destino, "Sem Registro") as Aerodromo_de_Destino
    ,ifnull(Aerodromo_de_Origem, "Sem Origem") as Aerodromo_de_Origem
    ,ifnull(CLS, "Sem Registro") as CLSTeste
    ,Categoria_da_Aeronave
    ,Danos_a_Aeronave
    ,Data_da_Ocorrencia
    ,UF
from anac
""")

display(novodf)